<a href="https://colab.research.google.com/github/supriyameduri9/CMPE255_Assignments/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using any data set you want, implement colabs for the following ANN algorithms (see hints)

Write proper documentation in read.me file and colabs

a) LSH

b) exhaustive search

c) product quantization

d) trees and graphs

e) hnsw

Hint : (use as motivation - not exact copy paste)

https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6 (Links to an external site.)

https://github.com/eyaltrabelsi/my-notebooks/blob/master/Lectures/search_in_practice-approximate_nearest_neighbors/Approximate%20Nearest%20Neighbors.ipynb (Links to an external site.)

Hint :

colabs at : https://github.com/eyaltrabelsi/my-notebooks/tree/master/Lectures/search_in_practice-approximate_nearest_neighbors (Links to an external site.)

### Install Necessary Libraries

In [ ]:
pip install lightfm

     |████████████████████████████████| 310 kB 7.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705358 sha256=a9d921fff08bb6f2632d2e35ddaf7c5b6eb3548e8aa4c2503736fd4bc4ff0f2e
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [110]:
pip install nmslib==2.0.11

ERROR: Could not find a version that satisfies the requirement nmslib==2.0.11 (from versions: 1.6.3, 1.7, 1.7.1, 1.7.2, 1.7.3.1, 1.7.3.2, 1.7.3.3, 1.7.3.4, 1.7.3.6, 1.8, 1.8.1, 2.0.4, 2.0.5, 2.0.6, 2.1.1)
ERROR: No matching distribution found for nmslib==2.0.11


In [14]:
pip install faiss-cpu --no-cache

     |████████████████████████████████| 8.4 MB 5.9 MB/s 


In [43]:
pip install annoy

     |████████████████████████████████| 646 kB 7.5 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391660 sha256=3ef99d7af8e88002f0e2fbb6ea34f9de48d27af797422983df36383ca506050c
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


### Import Dependencies

Here I am using [Stack Exchange Network](https://making.lyst.com/lightfm/docs/datasets.html) dataset provided by LightFM to apply different ANN algorithms

In [117]:
from lightfm import LightFM
import pickle
from lightfm.datasets import fetch_stackexchange
import faiss
import annoy
import nmslib

In [ ]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)


In [8]:
stackexchange = fetch_stackexchange('crossvalidated',
                           test_set_fraction=0.1,
                           indicator_features=False,
                           tag_features=True)
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

In [10]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

#### Load the data from pickeled model

In [11]:
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
vectors = data["vector"]
names = data["name"]
data

{'name': array(['bayesian', 'prior', 'elicitation', ..., 'events', 'mutlivariate',
        'sample-variance'], dtype='<U50'),
 'vector': array([[ 0.16641644,  0.3901509 , -0.6988499 , ...,  0.53172237,
          0.26979873,  0.16564623],
        [-0.22441788, -0.01032948, -0.04911942, ...,  0.4292133 ,
         -0.22123186,  0.20221922],
        [ 0.34115985, -0.17385964,  0.17200977, ...,  0.08830049,
         -0.18342032, -0.1314393 ],
        ...,
        [ 0.16750446,  0.21263315, -0.27324823, ...,  0.05092465,
         -0.01612114, -0.13511845],
        [ 0.17518018, -0.02437438,  0.15058647, ...,  0.1925765 ,
         -0.06998837, -0.19130738],
        [ 0.19713715,  0.599983  , -0.38426286, ..., -0.36256272,
          0.4619656 ,  0.3476233 ]], dtype=float32)}

### Exhaustive Search 

This technique involves comparing each vector with every other vector , usually not very optimal because it will require linear query time to iterate over the entire dataset.

Here I used Faiss ( facebook library for efficient similarity search and clustering of dense vectors).

In [36]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [37]:
index = ExactIndex(data["vector"], data["name"])
index.build()

In [67]:
stack_vector, stack_name = data['vector'][45], data['name'][45]
simlar_stack_names = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_names}")

The most similar stack to overdispersion are:
* bayesian
* prior
* elicitation
* distributions
* normality
* software
* open-source
* statistical-significance
* machine-learning
* dataset


### Trees and Forest 

For tree based algorithms we usually construct forests (collection of trees) as their data structure by splitting the dataset into subsets

We use annoy library where number_of_trees — the number of binary trees we build, search_k — the number of binary trees we search for each point.

In [79]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in range(len(indices) - 1)]
        # print(indices)

In [62]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [64]:
stack_vector, stack_name = data['vector'][45], data['name'][45]
simlar_stack_names = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_names}")

The most similar stack to overdispersion are:
* bayesian
* prior
* elicitation
* distributions
* normality
* software
* open-source
* statistical-significance
* machine-learning
* dataset


### Product Quantization

Since in real time, we deal with large dataset, therefore linear search is an expensive task, hence we use quantization technique where we reduce dataset size by defining a function that fits the data into compact form

In [90]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vector, k=10):
        distances, indices = self.index.search(vectors, k) 
        print(indices)
        print(distances)
        return [self.labels[i] for i in range(len(indices[0]))]
        # return [self.labels[i] for i in range(len(indices))]

In [91]:
index = IVPQIndex(data["vector"], data["name"])
index.build()


In [92]:
stack_vector, stack_name = data['vector'][45], data['name'][45]
simlar_stack_names = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_names}")

[[ 2030  5045  9524 ... 16214 15880     0]
 [  250 49521 13369 ... 12005 11353 10352]
 [ 5813   433     2 ... 14397  7202 45018]
 ...
 [ 3600   618  5171 ...  6982  4852  3116]
 [72358 34145 68510 ... 27352  2294 31515]
 [11046 13304 12175 ... 24560 15912  7309]]
[[2.844661  2.844661  2.844661  ... 2.844661  2.844661  2.844661 ]
 [1.4752072 1.4752072 1.4752072 ... 1.510464  1.510464  1.510464 ]
 [4.156625  4.156625  4.156625  ... 4.4509544 4.471764  4.5681314]
 ...
 [2.2162719 2.2162719 2.2162719 ... 2.2162719 2.2162719 2.2162719]
 [2.994725  3.45093   3.45093   ... 4.4976053 4.4976053 4.4976053]
 [3.897081  3.897081  3.897081  ... 3.897081  3.897081  3.897081 ]]
The most similar stack to overdispersion are:
* bayesian
* prior
* elicitation
* distributions
* normality
* software
* open-source
* statistical-significance
* machine-learning
* dataset


### LSH - Local Sensitive Hashing

We use a hash table to map points nearby into the same bucket in this technique. For this we use faiss library, where num_bits — A larger value will give more accurate results, but larger indexes.



In [120]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in range(len(indices[0]))]

In [121]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [ ]:
stack_vector, stack_name = data['vector'][45], data['name'][45]
simlar_stack_names = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_names}")

### HSNW - Hierarchical Navigable Small World Graphs

In this algorithm, we reduce the search time by finding the average path. This Algorithm follows the below steps
 we  At each step 

1.   Start at a random entry point and iteratively traverse the graph.
2.   At each step, we find the distances from a query to the neighbors of a current base node and then selects as the next base node the adjacent node that minimizes the distance, while constantly keeping track of the best-discovered neighbors. 
3. The search is terminated when some stopping condition is met.




In [113]:
pip install nmslib

     |████████████████████████████████| 13.5 MB 6.0 MB/s 
     |████████████████████████████████| 188 kB 41.8 MB/s 


In [114]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in range(len(indices[0]))]

In [118]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [119]:
stack_vector, stack_name = data['vector'][45], data['name'][45]
simlar_stack_names = '\n* '.join(index.query(stack_vector))
print(f"The most similar stack to {stack_name} are:\n* {simlar_stack_names}")

The most similar stack to overdispersion are:
* bayesian
* prior
* elicitation
* distributions
* normality
* software
* open-source
* statistical-significance
* machine-learning
* dataset


### References

1.   https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6
2.   https://github.com/lyst/lightfm/blob/master/examples/stackexchange/hybrid_crossvalidated.ipynb

